In [46]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, ZeroPadding2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, BatchNormalization, Add, Input
from keras.callbacks import ReduceLROnPlateau
from keras.utils import to_categorical

In [60]:
X_train = np.load("features_train.npy")
X_test = np.load("features_test.npy")
y_train = np.load("labels_train.npy")
y_test = np.load("labels_test.npy")

X_train = X_train / 255.0
X_test = X_test / 255.0
y_train = to_categorical(y_train, 3)
y_test = to_categorical(y_test, 3)

X_val = X_train[-50:]
y_val = y_train[-50:]
X_train = X_train[:-50]
y_train = y_train[:-50]
IMG_SIZE = 224
DATADIR_TRAIN = "C:/Users/Gurpreet/Documents/BME571 - ML in BME/COVID_CNN/Covid19-dataset/train"
DATADIR_TEST = "C:/Users/Gurpreet/Documents/BME571 - ML in BME/COVID_CNN/Covid19-dataset/test"

In [61]:
print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(y_test.shape))
print ("X_val shape: " + str(X_val.shape))
print ("Y_val shape: " + str(y_val.shape))

number of training examples = 201
number of test examples = 66
X_train shape: (201, 224, 224, 3)
Y_train shape: (201, 3)
X_test shape: (66, 224, 224, 3)
Y_test shape: (66, 3)
X_val shape: (50, 224, 224, 3)
Y_val shape: (50, 3)


In [62]:
# input activation has the same dimension as the output activation.
def identity_block(X, kernel_size, filters):
    k = kernel_size
    # save the value to add later
    x_skip = X
    
    # Layer 1`
    Conv2D(filters, kernel_size = (k, k), strides = (1,1), padding = 'same')(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)

    # Layer 2
    X = Conv2D(filters, kernel_size = (k,k), strides = (1,1), padding = 'same')(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)


    # Add skip value to main path
    X = Add()([X, x_skip])
    X = Activation('relu')(X)
    
    
    return X

In [63]:
# input and output dimensions don’t match up. 
# The difference with the identity block is that there is a CONV2D layer in the shortcut path.
def convolution_block(X, filters, s = 2):
    # save the value to add later
    x_skip = X
    
    # Layer 1
    X = Conv2D(filters, (3,3), padding = 'same', strides = (s, s))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    
    # Layer 2
    X = Conv2D(filters, (3,3), padding = 'same')(X)
    X = BatchNormalization(axis=3)(X)
    
    # Processing skip with conv(1,1)
    x_skip = Conv2D(filters, (1,1), strides = (s,s))(x_skip)
    
    # Add skip to main path
    X = Add()([X, x_skip])     
    X = Activation('relu')(X)
    
    return X

In [64]:
'''
The identity
shortcuts (Eqn.(1)) can be directly used when the input and
output are of the same dimensions (solid line shortcuts in
Fig. 3). When the dimensions increase (dotted line shortcuts
in Fig. 3), we consider two options: (A) The shortcut still
performs identity mapping, with extra zero entries padded
for increasing dimensions. This option introduces no extra
parameter; (B) The projection shortcut in Eqn.(2) is used to
match dimensions (done by 1×1 convolutions). For both
options, when the shortcuts go across feature maps of two
sizes, they are performed with a stride of 2.
'''

'\nThe identity\nshortcuts (Eqn.(1)) can be directly used when the input and\noutput are of the same dimensions (solid line shortcuts in\nFig. 3). When the dimensions increase (dotted line shortcuts\nin Fig. 3), we consider two options: (A) The shortcut still\nperforms identity mapping, with extra zero entries padded\nfor increasing dimensions. This option introduces no extra\nparameter; (B) The projection shortcut in Eqn.(2) is used to\nmatch dimensions (done by 1×1 convolutions). For both\noptions, when the shortcuts go across feature maps of two\nsizes, they are performed with a stride of 2.\n'

In [65]:
def ResNet34(shape = (IMG_SIZE, IMG_SIZE, 3), classes = 3):
    X_input = Input(shape)
    # skipped max padding for now 
    #X = ZeroPadding2D((3, 3))(X_input)
    #initialize layer weight may be useful
    # STAGE 1: convolution w/ kernal size 7x7,  64 filters, with stride 2, followed by max pool
    X = Conv2D(64, kernel_size = 7, strides = 2, padding = 'same')(X_input)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3,3), strides=(2,2))(X)
    
    # STAGE 2
    X = convolution_block(X, filters = 64)
    X = identity_block(X, 3, 64)
    X = identity_block(X, 3, 64)

    # STAGE 3
    X = convolution_block(X, filters = 128)
    X = identity_block(X, 3, 128)
    X = identity_block(X, 3, 128)
    X = identity_block(X, 3, 128)
    
    # STAGE 4
    X = convolution_block(X, filters = 256)
    X = identity_block(X, 3, 256)
    X = identity_block(X, 3, 256)
    X = identity_block(X, 3, 256)
    X = identity_block(X, 3, 256)
    X = identity_block(X, 3, 256)
    
    # STAGE 5
    X = convolution_block(X, filters = 512)
    X = identity_block(X, 3, 512)
    X = identity_block(X, 3, 512)
    
    X = AveragePooling2D(2,2, padding='same')(X)
    
    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax')(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet34')
    return model

In [66]:
model = ResNet34(shape = (IMG_SIZE,IMG_SIZE, 3), classes = 3)

In [67]:
model.compile(
    optimizer='adam', # optimizer
    loss='categorical_crossentropy', # loss function to optimize 
    metrics=['accuracy'] # metrics to monitor
)

In [68]:
model.summary()

Model: "ResNet34"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_111 (Conv2D)            (None, 112, 112, 64  9472        ['input_4[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_99 (BatchN  (None, 112, 112, 64  256        ['conv2d_111[0][0]']             
 ormalization)                  )                                                          

                                                                                                  
 add_51 (Add)                   (None, 14, 14, 128)  0           ['batch_normalization_107[0][0]',
                                                                  'conv2d_121[0][0]']             
                                                                                                  
 activation_145 (Activation)    (None, 14, 14, 128)  0           ['add_51[0][0]']                 
                                                                                                  
 batch_normalization_108 (Batch  (None, 14, 14, 128)  512        ['activation_145[0][0]']         
 Normalization)                                                                                   
                                                                                                  
 activation_146 (Activation)    (None, 14, 14, 128)  0           ['batch_normalization_108[0][0]']
          

 batch_normalization_117 (Batch  (None, 7, 7, 256)   1024        ['conv2d_132[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_158 (Activation)    (None, 7, 7, 256)    0           ['batch_normalization_117[0][0]']
                                                                                                  
 add_56 (Add)                   (None, 7, 7, 256)    0           ['activation_158[0][0]',         
                                                                  'activation_156[0][0]']         
                                                                                                  
 activation_159 (Activation)    (None, 7, 7, 256)    0           ['add_56[0][0]']                 
                                                                                                  
 batch_nor

 activation_172 (Activation)    (None, 4, 4, 512)    0           ['batch_normalization_126[0][0]']
                                                                                                  
 conv2d_142 (Conv2D)            (None, 4, 4, 512)    2359808     ['activation_172[0][0]']         
                                                                                                  
 batch_normalization_127 (Batch  (None, 4, 4, 512)   2048        ['conv2d_142[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 conv2d_143 (Conv2D)            (None, 4, 4, 512)    131584      ['activation_171[0][0]']         
                                                                                                  
 add_61 (Add)                   (None, 4, 4, 512)    0           ['batch_normalization_127[0][0]',
          

In [69]:
model.fit(
    X_train, y_train,
    epochs = 5,
    batch_size = 32,
    validation_data=(X_val, y_val))

Epoch 1/5
7/7 [==============================] - 39s 5s/step - loss: 5.0829 - accuracy: 0.6219 - val_loss: 2723.6072 - val_accuracy: 0.2400
Epoch 2/5
7/7 [==============================] - 35s 5s/step - loss: 1.0412 - accuracy: 0.7910 - val_loss: 1309.4794 - val_accuracy: 0.2400
Epoch 3/5
7/7 [==============================] - 36s 5s/step - loss: 0.4033 - accuracy: 0.8209 - val_loss: 238.8468 - val_accuracy: 0.3000
Epoch 4/5
7/7 [==============================] - 36s 5s/step - loss: 0.1670 - accuracy: 0.9353 - val_loss: 5.3399 - val_accuracy: 0.5400
Epoch 5/5
7/7 [==============================] - 38s 5s/step - loss: 0.0977 - accuracy: 0.9652 - val_loss: 1.2963 - val_accuracy: 0.4200


In [70]:
preds = model.evaluate(X_test, y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

3/3 [==============================] - 1s 211ms/step - loss: 1.3724 - accuracy: 0.3939
Loss = 1.3724080324172974
Test Accuracy = 0.39393940567970276


In [71]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import random

In [72]:
from keras.utils import load_img, img_to_array

CATEGORIES = ["Covid", "Normal", "Viral Pneumonia"]
images = []
for category in CATEGORIES:
    path = os.path.join(DATADIR_TEST, category)
    #print(path)
    class_type = CATEGORIES.index(category)
    for img in os.listdir(path):
        img = cv2.imread(os.path.join(path, img))
        
        img = tf.image.resize(img, (IMG_SIZE,IMG_SIZE))
        #img = load_img(img, target_size=(IMG_SIZE,IMG_SIZE))
        img = img_to_array(img)
        img = np.expand_dims(img, axis=0)
        images.append(img)
#images = np.vstack(images)

In [73]:
#images = np.vstack(images)
images = np.array(images)
for i in range(0, len(images)):
    classes = model.predict(images[i])
    print(classes)

1/1 [==============================] - 1s 522ms/step
[[0. 0. 1.]]
1/1 [==============================] - 0s 44ms/step
[[0. 0. 1.]]
1/1 [==============================] - 0s 44ms/step
[[0. 0. 1.]]
1/1 [==============================] - 0s 46ms/step
[[0. 0. 1.]]
1/1 [==============================] - 0s 46ms/step
[[0. 0. 1.]]
1/1 [==============================] - 0s 47ms/step
[[0. 0. 1.]]
1/1 [==============================] - 0s 47ms/step
[[0. 0. 1.]]
1/1 [==============================] - 0s 44ms/step
[[0. 0. 1.]]
1/1 [==============================] - 0s 48ms/step
[[0. 0. 1.]]
1/1 [==============================] - 0s 44ms/step
[[0. 0. 1.]]
1/1 [==============================] - 0s 43ms/step
[[0. 0. 1.]]
1/1 [==============================] - 0s 46ms/step
[[0. 0. 1.]]
1/1 [==============================] - 0s 45ms/step
[[0. 0. 1.]]
1/1 [==============================] - 0s 48ms/step
[[0. 0. 1.]]
1/1 [==============================] - 0s 45ms/step
[[0. 0. 1.]]
1/1 [===================

In [74]:
predictions = model.predict(images[:65])

3/3 [==============================] - 1s 223ms/step


In [75]:
images = np.array(images)
images.shape

(66, 1, 224, 224, 3)

In [76]:
predictions

array([[0.        , 0.        , 0.99999994],
       [0.        , 0.        , 0.99999994],
       [0.        , 0.        , 0.99999994],
       [0.        , 0.        , 0.99999994],
       [0.        , 0.        , 0.99999994],
       [0.        , 0.        , 0.99999994],
       [0.        , 0.        , 0.99999994],
       [0.        , 0.        , 0.99999994],
       [0.        , 0.        , 0.99999994],
       [0.        , 0.        , 0.99999994],
       [0.        , 0.        , 0.99999994],
       [0.        , 0.        , 0.99999994],
       [0.        , 0.        , 0.99999994],
       [0.        , 0.        , 0.99999994],
       [0.        , 0.        , 0.99999994],
       [0.        , 0.        , 0.99999994],
       [0.        , 0.        , 0.99999994],
       [0.        , 0.        , 0.99999994],
       [0.        , 0.        , 0.99999994],
       [0.        , 0.        , 0.99999994],
       [0.        , 0.        , 0.99999994],
       [0.        , 0.        , 0.99999994],
       [0.